### 載入預設的庫(20200507MUST)
* 處理bz2
* 處理kv值
* 載入正規表示法
* 載入英文分詞庫
* numpy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import bz2
from collections import Counter
import re
import nltk
import numpy as np
nltk.download('punkt')

train_file = bz2.BZ2File('/content/drive/My Drive/train.ft.txt.bz2')
test_file = bz2.BZ2File('/content/drive/My Drive/test.ft.txt.bz2')

train_file = train_file.readlines()
test_file = test_file.readlines()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 看一下檔案中有什麼，並且看一共有多大

In [3]:
print(train_file[67])
print(len(train_file))


b'__label__2 Even Mommy has fun with this one!: My four year old daughter loves everything Barbie and loves the Rapunzel movie. This game is tons of fun, even for a 42 year old. We love playing it together. We love decorating all the rooms and finding the gems. What even better is, she can play it alone and I get some me time!\n'
3600000


### 原檔案太大，只取其中80萬，原來有360萬
### 同時將檔案轉成utf8格式

In [0]:
num_train = 800000  # We're training on the first 800,000 reviews in the dataset
num_test = 200000  # Using 200,000 reviews from test set
# num_train = len(train_file)
# num_test = len(test_file)
train_file = [x.decode('utf-8') for x in train_file[:num_train]]
test_file = [x.decode('utf-8') for x in test_file[:num_test]]


### 看一下檔案中有什麼，並且看一共有多大

In [5]:
print(train_file[67])
print(len(train_file))


__label__2 Even Mommy has fun with this one!: My four year old daughter loves everything Barbie and loves the Rapunzel movie. This game is tons of fun, even for a 42 year old. We love playing it together. We love decorating all the rooms and finding the gems. What even better is, she can play it alone and I get some me time!

800000


### 將標記抓出，正評為1、負評為0，同時對訓練集和測試集進行

In [0]:
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file]

test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]


### 看一下標記和資料的長相

In [7]:
print(len(train_labels))
print(train_labels[67])
print(train_sentences[67])


800000
1
even mommy has fun with this one!: my four year old daughter loves everything barbie and loves the rapunzel movie. this game is tons of fun, even for a 42 year old. we love playing it together. we love decorating all the rooms and finding the gems. what even better is, she can play it alone and i get some me time!


### 把數字全部變成0

In [0]:
# Some simple cleaning of data
for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])
    

### 找一筆資料出來看

In [9]:
print(train_sentences[67])

even mommy has fun with this one!: my four year old daughter loves everything barbie and loves the rapunzel movie. this game is tons of fun, even for a 00 year old. we love playing it together. we love decorating all the rooms and finding the gems. what even better is, she can play it alone and i get some me time!


### 把網址全部換成'\<url>'

In [0]:
# Modify URLs to <url>
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

### 找一筆出來看

In [11]:
with_url = 0
for ii, s in enumerate(train_sentences):
    if '<url>' in s:
        with_url = ii
        print(ii)
        break
print(train_file[5])
print(train_sentences[with_url])

5
__label__2 an absolute masterpiece: I am quite sure any of you actually taking the time to read this have played the game at least once, and heard at least a few of the tracks here. And whether you were aware of it or not, Mitsuda's music contributed greatly to the mood of every single minute of the whole game.Composed of 3 CDs and quite a few songs (I haven't an exact count), all of which are heart-rendering and impressively remarkable, this soundtrack is one I assure you you will not forget. It has everything for every listener -- from fast-paced and energetic (Dancing the Tokage or Termina Home), to slower and more haunting (Dragon God), to purely beautifully composed (Time's Scar), to even some fantastic vocals (Radical Dreamers).This is one of the best videogame soundtracks out there, and surely Mitsuda's best ever. ^_^

an absolute masterpiece: i am quite sure any of you actually taking the time to read this have played the game at least once, and heard at least a few of the tr

### 將句子分詞為單字
* 啟用Counter
* 先將某條句定清空，再將分詞完之後的單詞存入該句子，使用ntlk

In [12]:
words = Counter()  # Dictionary that will map a word to the number of times it appeared in all the training sentences
for i, sentence in enumerate(train_sentences):
    # The sentences will be stored as a list of words/tokens
    train_sentences[i] = []
    for word in nltk.word_tokenize(sentence):  # Tokenizing the words
        words.update([word.lower()])  # Converting all the words to lowercase
        train_sentences[i].append(word)
    if i%20000 == 0:
        print(str((i*100)/num_train) + "% done")
print("100% done")

0.0% done
2.5% done
5.0% done
7.5% done
10.0% done
12.5% done
15.0% done
17.5% done
20.0% done
22.5% done
25.0% done
27.5% done
30.0% done
32.5% done
35.0% done
37.5% done
40.0% done
42.5% done
45.0% done
47.5% done
50.0% done
52.5% done
55.0% done
57.5% done
60.0% done
62.5% done
65.0% done
67.5% done
70.0% done
72.5% done
75.0% done
77.5% done
80.0% done
82.5% done
85.0% done
87.5% done
90.0% done
92.5% done
95.0% done
97.5% done
100% done


### 查看分完詞後的句子長相

In [13]:
print(train_sentences[67])


['even', 'mommy', 'has', 'fun', 'with', 'this', 'one', '!', ':', 'my', 'four', 'year', 'old', 'daughter', 'loves', 'everything', 'barbie', 'and', 'loves', 'the', 'rapunzel', 'movie', '.', 'this', 'game', 'is', 'tons', 'of', 'fun', ',', 'even', 'for', 'a', '00', 'year', 'old', '.', 'we', 'love', 'playing', 'it', 'together', '.', 'we', 'love', 'decorating', 'all', 'the', 'rooms', 'and', 'finding', 'the', 'gems', '.', 'what', 'even', 'better', 'is', ',', 'she', 'can', 'play', 'it', 'alone', 'and', 'i', 'get', 'some', 'me', 'time', '!']


### 把只出現一次的字刪除，只留出現兩次以上的字
### 把字照出現次數排序

In [0]:
words = {k:v for k,v in words.items() if v>1}
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)


### 印出一個字來看長什麼樣子

In [15]:
print(len(words))
print(words[100])

225962
she


### 將未知字和補0算一個字加進去
* 開始將字編號
* 建立字對編號的關係，雙向所以有兩個

In [0]:
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

### 印出某個字來看其索引，並且印出索引看字是什麼

In [17]:
print(word2idx['the'])
print(idx2word[100])



3
album


### 把訓練句子中的所有文字都轉成索引，還有測試句子

In [0]:
for i, sentence in enumerate(train_sentences):
    # Looking up the mapping dictionary and assigning the index to the respective words
    train_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

for i, sentence in enumerate(test_sentences):
    # For test sentences, we have to tokenize the sentences as well
    test_sentences[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]

### 印出一個句子出來看看

In [19]:
print(len(train_sentences[67]))
print(train_sentences[67])

71
[88, 6044, 58, 238, 23, 11, 31, 15, 13, 28, 610, 223, 152, 462, 483, 309, 4718, 6, 483, 3, 44246, 50, 2, 11, 211, 12, 2607, 10, 238, 4, 88, 16, 7, 81, 223, 152, 2, 86, 89, 487, 9, 356, 2, 86, 89, 8389, 35, 3, 4865, 6, 1288, 3, 4155, 2, 57, 88, 109, 12, 4, 102, 69, 262, 9, 788, 6, 5, 71, 79, 59, 70, 15]


### 印一個超過200的句子來看

In [20]:
more_than_200 = 0
for ii, s in enumerate(train_sentences):
    if len(s) > 200:
        more_than_200 = ii
        print(len(s))
        print(ii)
        break
print(train_file[more_than_200])
print(train_sentences[more_than_200])

204
71
__label__1 barbie rapunzel = crying child: My 6-year old daughter is currently sobbing in her bedroom as a result of this rotten game.She spent an hour on this game painting in pretty pictures and coloring in flowers. But halfway through the game, the program hung, and wouldn't let us leave the room or click anywhere else. Now even if we exit the game and then start it back up, we are frozen in one spot while Barbie's disembodied voice urges us to "explore somewhere else in the castle."Standard software troubleshooting and rebooting didn't help. Vivendi Games' support site is "currently disabled." And now I'm left with a weeping, frustrating child who doesn't understand that it is not her fault that her game won't work. I'm hoping it was just incompetent software programming that couldn't catch the bug my 6-year old caught the first time around. Or maybe they just don't care about the problems, so long as you get our 20 bucks. If you don't want a crying little girl, avoid this g

### 把句子不到200的前面補上0，如果超過200，就把200以後的刪除，讓每個句子的長度都是200

In [0]:
# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

seq_len = 200  # The length that the sentences will be padded/shortened to

train_sentences = pad_input(train_sentences, seq_len)
test_sentences = pad_input(test_sentences, seq_len)

# Converting our labels into numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

### 印一個補過0的出來看

In [22]:
print(len(train_sentences[67]))
print(train_sentences[67])

200
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0    88  6044    58
   238    23    11    31    15    13    28   610   223   152   462   483
   309  4718     6   483     3 44246    50     2    11   211    12  2607
    10   238     4    88    16     7    81   22

### 印出一個被截斷的來看

In [23]:
print(len(train_sentences[more_than_200]))
print(train_sentences[more_than_200])

200
[ 4718 44246  2181  3093   466    13    28  6213   152   462    12  2038
 24705    14    85  3480    29     7  1278    10    11  6106     0   726
    52   810    25    11   211  3958    14   249   582     6  7678    14
  5047     2    22  2704   161     3   211     4     3   937  4344     4
     6    51    26   308   248   708     3   649    53  2822  1382   363
     2   129    88    39    86  7734     3   211     6   114   395     9
   137    74     4    86    30  4663    14    31  1927   176  4718    21
 43061   392 20866   248     8    33  3582  1523   363    14     3  4032
     2    33   981   757  8824     6 17960    76    26   325     2 28578
   792   155   605  1214    12    33  2038  9273     2    32     6   129
     5   122   388    23     7 20278     4  1697   466    80    82    26
   341    17     9    12    20    85  1863    17    85   211   350    26
   117     2     5   122   784     9    18    47  9773   757  2602    17
    93    26  1661     3  4006    28  6213   15

### 把測試資料集分成test和validation

In [0]:
split_frac = 0.5 # 50% validation, 50% test
split_id = int(split_frac * len(test_sentences))
val_sentences, test_sentences = test_sentences[:split_id], test_sentences[split_id:]
val_labels, test_labels = test_labels[:split_id], test_labels[split_id:]

### 看測試資料集的長度為多少

In [25]:
print(len(val_sentences))

100000


### 將資料載入pytorch格式
* 三個資料集的載入
* 設定批次大小
* 建立dataloader的generator

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))
test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

batch_size = 400

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

### 檢查是否有gpu存在

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### 建立網路
* `vocab_size`：字彙檔的大小，就是一個one-hot的資料
* `output_size`：大小為1，因為只要一個值，介於0和1之間
* `embedding_dim`：就是word2vec的特徵大小，從`vocab_size`降維過來的
* `hidden_dim`：hidden和cell states的維度
* `n_layers`：lstm的層數
* `drop_prob`：dropout的比例


In [0]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden
    

### 定義超參數
* 定義超參數
* 建立模型
* 將模型移入gpu中
* 定義學習率
* 定義損失函數
* 定義最佳化方法

In [0]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)

lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


## 接下來把整個網路手動走一遍

### 啟始化hidden和cell states的值

* 由於model已經移入gpu，因此呼叫model的函數產生的值也會在gpu中，就會出現cuda device
* 另外要建立tuple，每一個批次都要有自己的tuple，不是一個大tuple丟去就好了

In [30]:
hh = model.init_hidden(batch_size)
hh = tuple([e.data for e in hh])
print(hh[1].shape)
print(hh[1])

torch.Size([2, 400, 512])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')


In [31]:
inputs = next(iter(train_loader)) # 取一段training data出來
x = inputs[0].to(device) #把資料移入gpu
batch_num = x.size(0) #取得批次大小，256
x = x.long() # 把輸入值轉成長整數
print(x.shape) # 看看x的型狀長什麼樣子
print(x) #印出一個x來看看

torch.Size([400, 200])
tensor([[     0,      0,      0,  ...,      7, 142335,      2],
        [     0,      0,      0,  ...,      3,    144,      2],
        [     0,      0,      0,  ...,    189,    390,      2],
        ...,
        [     0,      0,      0,  ...,  48831,   3779,      2],
        [     0,      0,      0,  ...,   2761,     15,     44],
        [     0,      0,      0,  ...,    154,      9,      2]],
       device='cuda:0')


In [0]:
embedding = nn.Embedding(vocab_size, embedding_dim) # 把x從one-hot的詞彙大小轉成word2vec的大小
embedding = embedding.to(device) #將這個函數放入gpu

In [33]:
embeds = embedding(x) # 將詞彙轉成wordvec
embeds.shape #看看型狀

torch.Size([400, 200, 400])

In [0]:
lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=0.5, batch_first=True) # 建立一個lstm
lstm = lstm.to(device) #將lstm移入gpu
lstm_out, hidden = lstm(embeds, hh) #讓剛才的資料跑過lstm，產生輸出值及新的hidden值

In [35]:
print(lstm_out.shape) #看看從lstm跑出來的東西型狀
print(hidden[0].shape) # 看看hidden第一個值的形狀

torch.Size([400, 200, 512])
torch.Size([2, 400, 512])


In [36]:
lstm_out = lstm_out.contiguous().view(-1, hidden_dim) #把lstm的輸出值攤平
lstm_out.shape #看看形狀

torch.Size([80000, 512])

In [0]:
dropout = nn.Dropout(0.5) #設定dropout
out = dropout(lstm_out) # 讓輸出值走一次dropout

In [38]:
print(out.shape) # 查看輸出值的形狀

torch.Size([80000, 512])


In [39]:
fc = nn.Linear(hidden_dim, 1) #定義最後一層fc
fc = fc.to(device) #移入gpu
out = fc(out) #讓輸入出值走一次fc
print(out.shape) # 看一下形狀

torch.Size([80000, 1])


In [0]:
sigmoid = nn.Sigmoid() # 讓輸出值介於0和1之間
sigmoid = sigmoid.to(device) #將sigmoid移入gpu

In [41]:
out = sigmoid(out) #讓輸出值走一次sigmoid
print(out.shape) #看看形狀

torch.Size([80000, 1])


In [42]:
out = out.view(batch_size, -1) #把輸出值還原成(批次大小，句子長度)
print(out.shape) # 看形狀

torch.Size([400, 200])


In [43]:
out = out[:,-1] #取出最後結果值
print(out.shape)# 看形狀

torch.Size([400])


In [44]:
print(out) # 看每批256個輸出值

tensor([0.5062, 0.5036, 0.5103, 0.4936, 0.5039, 0.5121, 0.5130, 0.5062, 0.5049,
        0.5048, 0.5004, 0.5207, 0.4992, 0.5129, 0.4972, 0.5066, 0.4985, 0.4925,
        0.5159, 0.5044, 0.5165, 0.5119, 0.5102, 0.5034, 0.5075, 0.5085, 0.5088,
        0.4977, 0.4963, 0.5026, 0.4892, 0.5179, 0.5062, 0.5105, 0.5080, 0.4981,
        0.4959, 0.5028, 0.5025, 0.4956, 0.5026, 0.5002, 0.5100, 0.5035, 0.5131,
        0.5027, 0.5072, 0.5010, 0.5082, 0.5081, 0.4960, 0.4962, 0.4893, 0.5100,
        0.5213, 0.5023, 0.5109, 0.5091, 0.5106, 0.4984, 0.5096, 0.5094, 0.5005,
        0.5018, 0.5076, 0.4977, 0.5073, 0.4968, 0.5119, 0.5099, 0.4980, 0.5176,
        0.5146, 0.5014, 0.5145, 0.5100, 0.5065, 0.5070, 0.5044, 0.5156, 0.5126,
        0.5025, 0.4948, 0.4938, 0.5014, 0.5040, 0.5135, 0.4993, 0.5065, 0.5133,
        0.5189, 0.5043, 0.5048, 0.5126, 0.5074, 0.4954, 0.4925, 0.5121, 0.4987,
        0.5081, 0.5014, 0.5090, 0.5013, 0.4963, 0.5045, 0.5027, 0.5025, 0.5040,
        0.5080, 0.5121, 0.4967, 0.5176, 

### 接下來進行真正訓練
* 輪數為2
* 計數值
* 每1000步就印出結果
* 一開始的loss值為無限大

### 開始訓練
* 初始化hidden值
* 走訓練流程
* 每1000步就印出結果一次

In [45]:
epochs = 2
counter = 0
print_every = 1000
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

Epoch: 1/2... Step: 1000... Loss: 0.133544... Val Loss: 0.177070
Validation loss decreased (inf --> 0.177070).  Saving model ...
Epoch: 1/2... Step: 2000... Loss: 0.154081... Val Loss: 0.168578
Validation loss decreased (0.177070 --> 0.168578).  Saving model ...
Epoch: 2/2... Step: 3000... Loss: 0.192968... Val Loss: 0.171099
Epoch: 2/2... Step: 4000... Loss: 0.184609... Val Loss: 0.172703


### 進行模型結果預測

In [46]:
# Loading the best model
model.load_state_dict(torch.load('./state_dict.pt'))

test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze())  # Rounds the output to 0/1
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Test loss: 0.162
Test accuracy: 93.778%
